In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from PIL import Image
import os

from sklearn.preprocessing import StandardScaler
from time import time
from sklearn.utils.fixes import loguniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import ConfusionMatrixDisplay

In [2]:
folder="nimageType="" # jpg or jpgCrop
cropText=""  # Crop or NoCrop
modelName="" #Shorten the Name A bit


In [3]:
folderTrain= folder + 'Train' + imageType
X_train=[]
y_train=[]
imageSize=(224,224)
for i in os.listdir(folderTrain):
    X_train.append(np.array(Image.open(folderTrain+i).resize(imageSize)))
    y_train.append(i[:5])
X_train=np.array(X_train)[:,:,:,0].reshape(1008,224*224)
print(y_train[:3])

['FL_00', 'FL_00', 'FL_00']


In [4]:
X_train.shape


(1008, 50176)

In [5]:
X_train, X_val, y_train, y_val  = train_test_split(X_train,y_train,test_size = (208/1008),random_state=13,stratify=y_train)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [6]:
y_train_r = [x[-2:] for x in y_train]
y_train_f = [x[:2] for x in y_train]
y_val_r = [x[-2:] for x in y_val]
y_val_f = [x[:2] for x in y_val]
print(y_val_r[:5])
print(y_val_f[:5])

['27', '27', '27', '00', '27']
['NF', 'NF', 'NF', 'FL', 'NF']


In [7]:
target_names=['FL_00','FL_09','FL_18','FL_27','NF_00','NF_09','NF_18','NF_27']

In [8]:
import copy
def imageRotationCorrection(images,orientations):
    images=copy.deepcopy(images)
    for image in range(len(images)):
        if orientations[image][-2:] != '00':
            #print(f"{keys[r]}, {cnt[keys[r]]}, {rot:<30}")
            if(orientations[image][-2:])==  '09':
                images[image] = np.rot90(images[image])
            elif(orientations[image][-2:])==  '18':
                images[image]=np.rot90(images[image],k=2)
            elif(orientations[image][-2:])==  '27':
                images[image]=np.rot90(images[image],k=3)
    return images

In [9]:
def PCAEigen(min_components, max_components, step_value, X_train, X_val, y_train, y_val):
    best_fit = [0,0,0,0]
    for n_components in range(min_components, max_components,step_value):
    
        t0 = time()
        pca = PCA(n_components=n_components, svd_solver="randomized", whiten=True).fit(X_train)
    
        eigenfaces = pca.components_.reshape((n_components, 224, 224))

    
        t0 = time()
        X_train_pca = pca.transform(X_train)
        X_val_pca = pca.transform(X_val)
        t0 = time()
        param_grid = {
            "C": loguniform(1e3, 1e4, 1e5),
            "gamma": loguniform(1e-4, 1e-3, 1e-1),
            
            }
        clf = RandomizedSearchCV(
            SVC(kernel = "rbf", class_weight="balanced"), param_grid, n_iter=10
            )
        clf = clf.fit(X_train_pca, y_train)
        
        

       
        t0 = time()
        y_pred = clf.predict(X_val_pca)
        if best_fit[-1] <  accuracy_score(y_val, y_pred):
            best_fit=[n_components,clf.best_params_["C"],clf.best_params_["gamma"],accuracy_score(y_val, y_pred)]
        print(n_components, clf.best_estimator_,  accuracy_score(y_val, y_pred))
    return best_fit
        #print(n_components, accuracy_score(y_val, y_pred))
#ConfusionMatrixDisplay.from_estimator(
#    clf, X_val_pca, y_val, display_labels=target_names, xticks_rotation="vertical"
#)
#plt.tight_layout()
#plt.show()

In [10]:
min_start=50
top_start=150

mid_r = PCAEigen(min_start,top_start,25,X_train, X_val, y_train_r, y_val_r)[0]

bot= mid_r - 15
top= mid_r + 15

if min_start==mid_r: 
    bot = mid_r-20
    top = mid_r+10
    
best_r=PCAEigen(bot,top,2,X_train, X_val, y_train_r, y_val_r)

50 SVC(C=104357.87400957865, class_weight='balanced', gamma=0.10048505650452315) 0.8990384615384616
75 SVC(C=105870.76237792481, class_weight='balanced', gamma=0.10062258787364671) 0.8701923076923077
100 SVC(C=103591.54677192452, class_weight='balanced', gamma=0.10027564079257681) 0.7692307692307693
125 SVC(C=103142.75193635192, class_weight='balanced', gamma=0.10060935558208771) 0.7211538461538461
30 SVC(C=106089.0966216026, class_weight='balanced', gamma=0.1008395132011176) 0.9375
32 SVC(C=102047.39545928781, class_weight='balanced', gamma=0.1003168791957864) 0.9471153846153846
34 SVC(C=102289.99124908705, class_weight='balanced', gamma=0.10012301192773897) 0.9519230769230769
36 SVC(C=102679.74055195203, class_weight='balanced', gamma=0.10087536807760879) 0.9423076923076923
38 SVC(C=101303.67511293155, class_weight='balanced', gamma=0.10010481547088759) 0.9519230769230769
40 SVC(C=102770.2904592885, class_weight='balanced', gamma=0.10018035663560815) 0.9278846153846154
42 SVC(C=10469

In [11]:
n_components=best_r[0]
print(
    "Extracting the top %d eigenradiographs from %d radiographs" % (n_components, X_train.shape[0])
)
t0 = time()
pca_r = PCA(n_components=n_components, svd_solver="randomized", whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

eigenfaces = pca_r.components_.reshape((n_components, 224, 224))

print("Projecting the input data on the eigenradiographs orthonormal basis")
t0 = time()
X_train_pca_r = pca_r.transform(X_train)
X_val_pca_r = pca_r.transform(X_val)
print("done in %0.3fs" % (time() - t0))


Extracting the top 34 eigenradiographs from 800 radiographs
done in 2.472s
Projecting the input data on the eigenradiographs orthonormal basis
done in 0.246s


In [12]:
classifier_r = SVC(kernel = "rbf",C=best_r[1], gamma=best_r[2], class_weight="balanced").fit(X_train_pca_r, y_train_r)
        

In [ ]:
min_start=50
top_start=150

mid_f = PCAEigen(min_start,top_start,25,X_train, X_val, y_train_f, y_val_f)[0]
top= mid_f - 15
top= mid_f + 15

if min_start==mid_f:
    bot = mid_f-20
    top = mid_f+10
    
best_f=PCAEigen(bot,mid_f+15,2,X_train, X_val, y_train_f, y_val_f)

50 SVC(C=101723.1060511849, class_weight='balanced', gamma=0.10014292843566418) 0.5913461538461539
75 SVC(C=105229.01194945748, class_weight='balanced', gamma=0.1003525641857057) 0.6057692307692307
100 SVC(C=102230.91847445742, class_weight='balanced', gamma=0.10041296166429814) 0.625
125 SVC(C=108368.2271102692, class_weight='balanced', gamma=0.10012306133338923) 0.6105769230769231
30 SVC(C=104142.39716493536, class_weight='balanced', gamma=0.10048658395883636) 0.5721153846153846
32 SVC(C=101207.77583078943, class_weight='balanced', gamma=0.1009514552116489) 0.5913461538461539
34 SVC(C=101102.94657765243, class_weight='balanced', gamma=0.10029717739857512) 0.5528846153846154
36 SVC(C=103342.66406874724, class_weight='balanced', gamma=0.10026705759362604) 0.5625
38 SVC(C=101385.42264027633, class_weight='balanced', gamma=0.10014636673306843) 0.5576923076923077
40 SVC(C=104040.9600089575, class_weight='balanced', gamma=0.10029554106710925) 0.5673076923076923
42 SVC(C=101028.80302806878,

In [ ]:
n_components=best_f[0]
print(
    "Extracting the top %d eigenradiographs from %d radiographs" % (n_components, X_train.shape[0])
)
t0 = time()
pca_f = PCA(n_components=n_components, svd_solver="randomized", whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

eigenfaces = pca_f.components_.reshape((n_components, 224, 224))

print("Projecting the input data on the eigenradiographs orthonormal basis")
t0 = time()
X_train_pca_f = pca_f.transform(X_train)
X_val_pca_f = pca_f.transform(X_val)
print("done in %0.3fs" % (time() - t0))


In [ ]:
classifier_f = SVC(kernel = "rbf",C=best_f[1], gamma=best_f[2], class_weight="balanced").fit(X_train_pca_f, y_train_f)

In [ ]:
folderTest= folder + 'Test' + imageType
X_test=[]
y_test=[]
imageSize=(224,224)
for i in os.listdir(folderTest):
    X_test.append(np.array(Image.open(folderTest+i).resize(imageSize)))
    y_test.append(i[:5])
test_size=len(X_test)
X_test=np.array(X_test)[:,:,:,0].reshape(test_size,224*224)
X_test = scaler.transform(X_test)
X_test_pca_r = pca_r.transform(X_test)

X_test_pca_f = pca_f.transform(X_test)

In [ ]:
y_test_r = [x[-2:] for x in y_test]
y_test_f = [x[:2] for x in y_test]

In [ ]:
print(test_size)

In [ ]:
r_names=['00','09','18','27']
f_names=['FL','NF']

In [ ]:
y_pred_r = classifier_r.predict(X_test_pca_r)
print(best_r[0], accuracy_score(y_test_r, y_pred_r))
ConfusionMatrixDisplay.from_estimator(
    classifier_r, X_test_pca_r, y_test_r, display_labels=r_names, xticks_rotation="vertical"
)
plt.tight_layout()
plt.show()

In [ ]:
y_pred_f = classifier_f.predict(X_test_pca_f)
print(best_f[0], accuracy_score(y_test_f, y_pred_f))
ConfusionMatrixDisplay.from_estimator(
    classifier_f, X_test_pca_f, y_test_f, display_labels=f_names, xticks_rotation="vertical"
)
plt.tight_layout()
plt.show()